In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import fitsio

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from dataloc import *

In [ ]:
# CREATE BINS FOR WP CALCULATION
edges = np.geomspace(0.1, 20, 16)

# For my own use
with open(WP_RADIAL_BINS_DESI_FILE, 'w') as f:
    for i in range(len(edges)-1):
        f.write(f'{edges[i]:.8f} {edges[i+1]:.8f}\n')

# For xirunpc.py script
with open(WP_RADIAL_EDGE_DESI_FILE, 'w') as f:
    for i in range(len(edges)):
        f.write(f'{edges[i]:.8f}\n')

In [ ]:
def check_cols_for_tbl(path, colname):
    if os.path.isfile(path) == False:
        print(f'{path} not found')
        return
    cols = fitsio.FITS(path)[1].get_colnames()
    if colname not in cols:
        print(f'{colname} not in {path}')
    else:
        print(f'{colname} found in {path}')
    print(cols)
check_cols_for_tbl('/dvs_ro/cfs/cdirs/desi/survey/catalogs/Y1/LSS/iron/LSScats/v1.5pip/BGS_BRIGHT_full_HPmapcut.dat.fits', 'QUIESCENT')
check_cols_for_tbl('/global/cfs/cdirs/desi/users/ianw89/clustering/Y1/LSS/iron/LSScats/v1.5pip/BGS_BRIGHT_NGC_clustering.dat.fits', 'QUIESCENT')
check_cols_for_tbl('/global/cfs/cdirs/desi/users/ianw89/clustering/Y1/LSS/iron/LSScats/v1.5pip/BGS_BRIGHT_NGC_0_clustering.ran.fits', 'QUIESCENT')

In [ ]:
def get_wp_for(tracer, weights, survey, verspec, ver, bins, zmax, nran, quiescent=None):
    dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER, survey, 'LSS', verspec, 'LSScats', ver, 'rppi')

    if not os.path.exists(dir):
        print("WARNING: Directory does not exist: ", dir)
        return None

    if quiescent is not None:
        addon = '_QUIESCENT' + ('1' if quiescent else '0')
    else:
        addon = ''
        
    wp_fn = os.path.join(dir, f'wp_{tracer}_GCcomb_0.001_{zmax}_{weights}_{bins}_njack0_nran{nran}_split20{addon}.txt')

    if not os.path.exists(wp_fn):
        print("WARNING: File does not exist: ", wp_fn)
        return None

    return np.loadtxt(wp_fn)

def get_allcounts_for(weights, survey, verspec, ver, zmax, quiescent=None):
    dir = os.path.join(CUSTOM_CLUSTERING_RESULTS_FOLDER, survey, 'LSS', verspec, 'LSScats', ver, 'rppi')

    if not os.path.exists(dir):
        print("WARNING: Directory does not exist: ", dir)
        return None

    if quiescent is not None:
        addon = '_QUIESCENT' + ('1' if quiescent else '0')
    else:
        addon = ''
        
    allcounts_fn = os.path.join(dir, f'allcounts_BGS_BRIGHT_GCcomb_0.001_{zmax}_{weights}_log_njack0_nran8_split20{addon}.npy')

    if not os.path.exists(allcounts_fn):
        print("WARNING: File does not exist: ", allcounts_fn)
        return None

    return np.load(allcounts_fn, allow_pickle=True)

In [ ]:
BIN_DIVISOR = 1
ZMAX = 0.14977

In [ ]:
test = get_allcounts_for('default_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', ZMAX)
# list attributes of the object
type(test)
for key in test.item().keys():
    print(key)

In [ ]:
plt.figure()
tracer = "BGS_ANY"    
red_pip_angular = get_wp_for(tracer, 'pip_bitwise', 'Y1', 'iron', 'v1.5pip', "custom", ZMAX, 8, True)
blue_pip_angular = get_wp_for(tracer, 'pip_bitwise', 'Y1', 'iron', 'v1.5pip', "custom", ZMAX, 8, False)

#plt.plot(all_default[:,0], all_default[:,2], 'o', color='k')
plt.plot(red_pip_angular[:,0], red_pip_angular[:,2], 'o', color='r')
plt.plot(blue_pip_angular[:,0], blue_pip_angular[:,2], 'o', color='b')
#plt.plot(test3[:,1], test3[:,2], '.') #It's almost the same, what is the difference? TODO

plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$w_p(r_p)$')
plt.xlabel(r'$r_p$ [Mpc/h]') 


In [ ]:
tracer = 'BGS_BRIGHT'
all_default = get_wp_for(tracer, 'default_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8)
red_default = get_wp_for(tracer, 'default_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, True)
blue_default = get_wp_for(tracer, 'default_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, False)

red_default_angular = get_wp_for(tracer, 'default_angular_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, True)
blue_default_angular = get_wp_for(tracer, 'default_angular_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, False)

red_pip_angular = get_wp_for(tracer, 'pip_angular_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, True)
blue_pip_angular = get_wp_for(tracer, 'pip_angular_bitwise_FKP', 'Y1', 'iron', 'v1.5pip', 'log2', ZMAX,8, False)

plt.figure()
plt.plot(red_default[:,0], red_default[:,2], 'o', color='red', label='Red, default')
plt.plot(red_default_angular[:,0], red_default_angular[:,2], '-', color='darkred', label='Red, default angular')
plt.plot(red_pip_angular[:,0], red_pip_angular[:,2], '--', color='orange', label='Red, pip angular')
plt.plot(blue_default[:,0], blue_default[:,2], 'o', color='blue', label='Blue, default')
plt.plot(blue_default_angular[:,0], blue_default_angular[:,2], '-', color='darkblue', label='Blue, default angular')
plt.plot(blue_pip_angular[:,0], blue_pip_angular[:,2], '--', color='skyblue', label='Blue, pip angular')
plt.xscale('log')
plt.yscale('log')
plt.ylabel(r'$w_p(r_p)$')
plt.xlabel(r'$r_p$ [Mpc/h]') 
plt.legend()
plt.title('BGS BRIGHT -19<r<-18 ; z<0.14977')
plt.show()